In [1]:
#3 Create features 

In [2]:
#Possible features
#difference between character removal

#Gaan na die nuwe file kry die test data en run die code op dit ook maak die models dan om beter test scores te kry

In [3]:
import pandas as pd 
import numpy as np

import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('punkt')
from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import CountVectorizer

from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize, sent_tokenize
import textstat


from textblob import TextBlob

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\brand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

Read in the Training data as well as the test data

In [5]:
df=pd.read_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\Data_merged.parquet")
print(df.info())
df=df.sample(frac=1,random_state=42)#.head(5000)

<class 'pandas.core.frame.DataFrame'>
Index: 124754 entries, 0 to 126482
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   text    124754 non-null  object 
 1   label   124754 non-null  float64
 2   id      7503 non-null    float64
dtypes: float64(2), object(1)
memory usage: 3.8+ MB
None


In [6]:
df_test=pd.read_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\Data_merged_test.parquet")


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14583 entries, 0 to 14683
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14583 non-null  object
 1   label   14583 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 341.8+ KB


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 124754 entries, 96862 to 123657
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   text    124754 non-null  object 
 1   label   124754 non-null  float64
 2   id      7503 non-null    float64
dtypes: float64(2), object(1)
memory usage: 3.8+ MB


Word count fuction defined to calculate the word cound for each dataframe

In [9]:
%%time
# Vectorized function to calculate word count

word_count_func = np.vectorize(lambda text: len(word_tokenize(text)))
df['word_count'] = word_count_func(df['text'])


CPU times: total: 3min 5s
Wall time: 3min 7s


In [10]:
df.head()

,text,label,id,word_count
index,,,,
96862,redact brexit report spark new tugofwar uk par...,1.0,NaN,322
5061,Acting Attorney General Sally Yates refused to...,1.0,NaN,412
119057,Pakistan air ambulance helicopter crash kills ...,1.0,266.0,10
19524,0 Add Comment \nSOME people fail to plan their...,1.0,NaN,332
2538,Are you tired of supporting companies who are ...,1.0,NaN,391


In [11]:
%%time
df_test['word_count'] = word_count_func(df_test['text'])


CPU times: total: 9.78 s
Wall time: 9.91 s


In [12]:
'''
# Vectorized function to calculate average word length
avg_word_length_func = np.vectorize(lambda text: np.mean([len(word) for word in word_tokenize(text)]))
df['avg_word_length'] = avg_word_length_func(df['text'])
'''

"\n# Vectorized function to calculate average word length\navg_word_length_func = np.vectorize(lambda text: np.mean([len(word) for word in word_tokenize(text)]))\ndf['avg_word_length'] = avg_word_length_func(df['text'])\n"

Calculate the average sentence length

In [13]:
%%time
# Vectorized function to calculate average sentence length

avg_sentence_length_func = np.vectorize(lambda text: np.mean([len(word_tokenize(sentence)) for sentence in sent_tokenize(text)]))
df['avg_sentence_length'] = avg_sentence_length_func(df['text'])


D:\Programs\envs\FakeNewsResearch\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\Programs\envs\FakeNewsResearch\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


CPU times: total: 3min 42s
Wall time: 3min 43s


In [14]:
%%time
df_test['avg_sentence_length'] = avg_sentence_length_func(df_test['text'])


CPU times: total: 10.3 s
Wall time: 10.5 s


Determine the punctuation and capitalization frequency

In [15]:
%%time
# Vectorized function to calculate the frequency of punctuation marks

punctuation_chars = set('!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
df['text_punctuation_frequency'] = df['text'].apply(lambda text: sum(1 for char in text if char in punctuation_chars))


# Vectorized function to calculate the frequency of capitalized words or phrases
vectorized_calculate_capitalization_frequency = np.vectorize(lambda text: sum(1 for word in text.split() if word.isupper()))

df['text_capitalization_frequency'] = vectorized_calculate_capitalization_frequency(df['text'])



CPU times: total: 12.8 s
Wall time: 12.9 s


In [16]:
df.describe()
df.head(10)

,text,label,id,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency
index,,,,,,,
96862,redact brexit report spark new tugofwar uk par...,1.0,NaN,322,322.000000,0,0
5061,Acting Attorney General Sally Yates refused to...,1.0,NaN,412,27.466667,91,4
119057,Pakistan air ambulance helicopter crash kills ...,1.0,266.0,10,10.000000,5,0
19524,0 Add Comment \nSOME people fail to plan their...,1.0,NaN,332,27.666667,22,3
2538,Are you tired of supporting companies who are ...,1.0,NaN,391,27.928571,35,14
90690,whoa chris matthew defy liberal medium scripts...,0.0,NaN,133,133.000000,0,0
48857,BEIRUT (Reuters) - Hezbollah leader Sayyed Has...,0.0,NaN,375,31.250000,35,3
125612,@soonergrunt better than tornado!,0.0,9660.0,6,6.000000,2,0
76403,comment mob angry student surround student wan...,0.0,NaN,305,305.000000,0,0


In [17]:
%%time
punctuation_chars = set('!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
df_test['text_punctuation_frequency'] = df_test['text'].apply(lambda text: sum(1 for char in text if char in punctuation_chars))
df_test['text_capitalization_frequency'] = vectorized_calculate_capitalization_frequency(df_test['text'])


CPU times: total: 1.12 s
Wall time: 1.16 s


In [18]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 124754 entries, 96862 to 123657
Data columns (total 7 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   text                           124754 non-null  object 
 1   label                          124754 non-null  float64
 2   id                             7503 non-null    float64
 3   word_count                     124754 non-null  int32  
 4   avg_sentence_length            124751 non-null  float64
 5   text_punctuation_frequency     124754 non-null  int64  
 6   text_capitalization_frequency  124754 non-null  int32  
dtypes: float64(3), int32(2), int64(1), object(1)
memory usage: 6.7+ MB


In [19]:
%%time


df['text_before_character_removal']=df['text'].str.len()

def remove_non_characters(text):
    # Remove non-character characters using regular expressions
    cleaned_text = text.str.replace(r'[^a-zA-Z\s]', '', regex=True)
    return cleaned_text

# Apply the function to 'text' column
df['text'] = remove_non_characters(df['text'])

# Convert to lowercase
df['text'] = df['text'].str.lower()

df['text_after_character_removal']=df['text'].str.len()

df.describe()

CPU times: total: 3.72 s
Wall time: 3.79 s


,label,id,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal
count,124754.000000,7503.000000,124754.000000,124751.000000,124754.000000,124754.000000,124754.000000,124754.000000
mean,0.486846,5439.831401,452.419730,154.031939,38.297522,4.657903,2636.624421,2575.644597
std,0.499829,3141.748725,578.555587,255.439946,78.282398,12.025171,3089.549206,2977.097920
min,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000
25%,0.000000,2726.500000,152.000000,26.200000,0.000000,0.000000,983.000000,963.000000
50%,0.000000,5408.000000,316.500000,37.000000,8.000000,0.000000,1955.000000,1921.000000
75%,1.000000,8149.500000,556.000000,214.000000,53.000000,6.000000,3311.000000,3246.000000
max,1.000000,10873.000000,28009.000000,10210.000000,7295.000000,909.000000,142961.000000,129415.000000


In [20]:
%%time
df_test['text_before_character_removal']=df_test['text'].str.len()

def remove_non_characters(text):
    # Remove non-character characters using regular expressions
    cleaned_text = text.str.replace(r'[^a-zA-Z\s]', '', regex=True)
    return cleaned_text

# Apply the function to 'text' column
df_test['text'] = remove_non_characters(df_test['text'])

# Convert to lowercase
df_test['text'] = df_test['text'].str.lower()

df_test['text_after_character_removal']=df_test['text'].str.len()

df_test.describe()

CPU times: total: 203 ms
Wall time: 250 ms


,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal
count,14583.000000,14583.000000,14583.000000,14583.0,14583.0,14583.000000,14583.000000
mean,0.500240,310.965028,310.965028,0.0,0.0,2204.873277,2204.873277
std,0.500017,350.019227,350.019227,0.0,0.0,2459.186390,2459.186390
min,0.000000,1.000000,1.000000,0.0,0.0,3.000000,3.000000
25%,0.000000,139.000000,139.000000,0.0,0.0,986.000000,986.000000
50%,1.000000,236.000000,236.000000,0.0,0.0,1679.000000,1679.000000
75%,1.000000,387.000000,387.000000,0.0,0.0,2756.000000,2756.000000
max,1.000000,11936.000000,11936.000000,0.0,0.0,77745.000000,77745.000000


In [21]:
%%time
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    # Tokenize the text into individual words
    words = nltk.word_tokenize(text)
    
    # Remove stop words
    words = [word for word in words if word.lower() not in stop_words]
    
    # Perform stemming on each word
    words = [stemmer.stem(word) for word in words]
    
    # Join the processed words back into a single string
    processed_text = ' '.join(words)
    
    return processed_text

df['text_processed'] = ''

def process_row(row):
    text = row['text']
    processed_text = preprocess_text(text)

    
    return pd.Series([ processed_text])

df[['text_processed']] = df.apply(process_row, axis=1)

CPU times: total: 10min 21s
Wall time: 10min 26s


In [22]:
%%time
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    # Tokenize the text into individual words
    words = nltk.word_tokenize(text)
    
    # Remove stop words
    words = [word for word in words if word.lower() not in stop_words]
    
    # Perform stemming on each word
    words = [stemmer.stem(word) for word in words]
    
    # Join the processed words back into a single string
    processed_text = ' '.join(words)
    
    return processed_text

df_test['text_processed'] = ''

def process_row(row):
    text = row['text']
    processed_text = preprocess_text(text)
    return pd.Series([ processed_text])

df_test[['text_processed']] = df_test.apply(process_row, axis=1)

CPU times: total: 1min 15s
Wall time: 1min 17s


In [23]:
%%time
calculate_subjectivity_vectorized = np.vectorize(lambda text: TextBlob(text).sentiment.subjectivity, otypes=[float])

# Apply the vectorized function to create a 'subjectivity' column
df['subjectivity_text'] = calculate_subjectivity_vectorized(df['text_processed'])



CPU times: total: 1min 50s
Wall time: 1min 54s


In [24]:
%%time
# Apply the vectorized function to create a 'subjectivity' column
df_test['subjectivity_text'] = calculate_subjectivity_vectorized(df_test['text_processed'])


CPU times: total: 14.1 s
Wall time: 14.6 s


In [25]:
%%time
sia = SentimentIntensityAnalyzer()

# Apply sentiment analysis to 'text_processed' column
df['text_sentiment'] = [sia.polarity_scores(text)['compound'] for text in df['text_processed']]

df.head()

CPU times: total: 4min 54s
Wall time: 4min 58s


,text,label,id,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal,text_processed,subjectivity_text,text_sentiment
index,,,,,,,,,,,,
96862,redact brexit report spark new tugofwar uk par...,1.0,NaN,322,322.000000,0,0,2308,2308,redact brexit report spark new tugofwar uk par...,0.352900,-0.2263
5061,acting attorney general sally yates refused to...,1.0,NaN,412,27.466667,91,4,2431,2279,act attorney gener salli yate refus back donal...,0.405317,-0.9517
119057,pakistan air ambulance helicopter crash kills ...,1.0,266.0,10,10.000000,5,0,73,65,pakistan air ambul helicopt crash kill nine ht...,0.000000,-0.8126
19524,add comment \nsome people fail to plan their ...,1.0,NaN,332,27.666667,22,3,1485,1446,add comment peopl fail plan purchas use firewo...,0.493783,-0.7893
2538,are you tired of supporting companies who are ...,1.0,NaN,391,27.928571,35,14,2121,2078,tire support compani care less futur nation ti...,0.460303,0.7845


In [26]:
%%time
df_test['text_sentiment'] = [sia.polarity_scores(text)['compound'] for text in df_test['text_processed']]

df_test.head()

CPU times: total: 35.6 s
Wall time: 36 s


,text,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal,text_processed,subjectivity_text,text_sentiment
index,,,,,,,,,,,
0,trump promise new deal black america trump pro...,0,170,170.0,0,0,1187,1187,trump promis new deal black america trump prom...,0.547890,0.8658
1,foundation tie bedevil hillary clinton preside...,1,1087,1087.0,0,0,7994,7994,foundat tie bedevil hillari clinton presidenti...,0.310024,0.9666
2,number week long russia end oil dependence rbt...,0,55,55.0,0,0,352,352,number week long russia end oil depend rbth ye...,0.338095,0.0772
3,codesod rule ten remy porter remy escape enter...,0,124,124.0,0,0,1087,1087,codesod rule ten remi porter remi escap enterp...,0.261865,0.4404
4,switch chip know anymore home scitech switch c...,0,309,309.0,0,0,2141,2141,switch chip know anymor home scitech switch ch...,0.319414,0.9217


In [27]:
%%time
def calculate_reading_ease(text):
    if isinstance(text, float):
        return 0  # Return 0 for float values
    # Calculate the reading ease score
    reading_ease = textstat.flesch_reading_ease(text)
    return reading_ease

vectorized_calculate_reading_ease = np.vectorize(calculate_reading_ease, otypes=[float])

# Assuming you have a DataFrame 'df' with 'title_processed' and 'text_processed' columns
df['text_reading_ease'] = vectorized_calculate_reading_ease(df['text_processed'])

df.head()

CPU times: total: 52.1 s
Wall time: 52.8 s


,text,label,id,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal,text_processed,subjectivity_text,text_sentiment,text_reading_ease
index,,,,,,,,,,,,,
96862,redact brexit report spark new tugofwar uk par...,1.0,NaN,322,322.000000,0,0,2308,2308,redact brexit report spark new tugofwar uk par...,0.352900,-0.2263,-255.36
5061,acting attorney general sally yates refused to...,1.0,NaN,412,27.466667,91,4,2431,2279,act attorney gener salli yate refus back donal...,0.405317,-0.9517,-172.81
119057,pakistan air ambulance helicopter crash kills ...,1.0,266.0,10,10.000000,5,0,73,65,pakistan air ambul helicopt crash kill nine ht...,0.000000,-0.8126,46.44
19524,add comment \nsome people fail to plan their ...,1.0,NaN,332,27.666667,22,3,1485,1446,add comment peopl fail plan purchas use firewo...,0.493783,-0.7893,-46.27
2538,are you tired of supporting companies who are ...,1.0,NaN,391,27.928571,35,14,2121,2078,tire support compani care less futur nation ti...,0.460303,0.7845,-118.34


In [28]:
%%time
df_test['text_reading_ease'] = vectorized_calculate_reading_ease(df_test['text_processed'])

df_test.head()

CPU times: total: 5.48 s
Wall time: 5.75 s


,text,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal,text_processed,subjectivity_text,text_sentiment,text_reading_ease
index,,,,,,,,,,,,
0,trump promise new deal black america trump pro...,0,170,170.0,0,0,1187,1187,trump promis new deal black america trump prom...,0.547890,0.8658,-92.62
1,foundation tie bedevil hillary clinton preside...,1,1087,1087.0,0,0,7994,7994,foundat tie bedevil hillari clinton presidenti...,0.310024,0.9666,-1030.82
2,number week long russia end oil dependence rbt...,0,55,55.0,0,0,352,352,number week long russia end oil depend rbth ye...,0.338095,0.0772,32.57
3,codesod rule ten remy porter remy escape enter...,0,124,124.0,0,0,1087,1087,codesod rule ten remi porter remi escap enterp...,0.261865,0.4404,-88.23
4,switch chip know anymore home scitech switch c...,0,309,309.0,0,0,2141,2141,switch chip know anymor home scitech switch ch...,0.319414,0.9217,-242.17


In [29]:
%%time
def calculate_lexical_diversity(text):
    if isinstance(text, float):
        return 0  # Return 0 for float values
    
    # Tokenize the text into individual words
    words = text.split()
    
    # Calculate the lexical diversity score
    if len(words) > 0:
        lexical_diversity = len(set(words)) / len(words)
    else:
        lexical_diversity = 0
    
    return lexical_diversity

# Define a vectorized version of calculate_lexical_diversity
vectorized_calculate_lexical_diversity = np.vectorize(calculate_lexical_diversity, otypes=[float])

df['text_lexical_diversity'] = vectorized_calculate_lexical_diversity(df['text_processed'])

df.head()

CPU times: total: 3.44 s
Wall time: 3.64 s


,text,label,id,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal,text_processed,subjectivity_text,text_sentiment,text_reading_ease,text_lexical_diversity
index,,,,,,,,,,,,,,
96862,redact brexit report spark new tugofwar uk par...,1.0,NaN,322,322.000000,0,0,2308,2308,redact brexit report spark new tugofwar uk par...,0.352900,-0.2263,-255.36,0.642857
5061,acting attorney general sally yates refused to...,1.0,NaN,412,27.466667,91,4,2431,2279,act attorney gener salli yate refus back donal...,0.405317,-0.9517,-172.81,0.758929
119057,pakistan air ambulance helicopter crash kills ...,1.0,266.0,10,10.000000,5,0,73,65,pakistan air ambul helicopt crash kill nine ht...,0.000000,-0.8126,46.44,1.000000
19524,add comment \nsome people fail to plan their ...,1.0,NaN,332,27.666667,22,3,1485,1446,add comment peopl fail plan purchas use firewo...,0.493783,-0.7893,-46.27,0.773050
2538,are you tired of supporting companies who are ...,1.0,NaN,391,27.928571,35,14,2121,2078,tire support compani care less futur nation ti...,0.460303,0.7845,-118.34,0.786096


In [30]:
%%time
df_test['text_lexical_diversity'] = vectorized_calculate_lexical_diversity(df_test['text_processed'])

df.head()

CPU times: total: 359 ms
Wall time: 439 ms


,text,label,id,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal,text_processed,subjectivity_text,text_sentiment,text_reading_ease,text_lexical_diversity
index,,,,,,,,,,,,,,
96862,redact brexit report spark new tugofwar uk par...,1.0,NaN,322,322.000000,0,0,2308,2308,redact brexit report spark new tugofwar uk par...,0.352900,-0.2263,-255.36,0.642857
5061,acting attorney general sally yates refused to...,1.0,NaN,412,27.466667,91,4,2431,2279,act attorney gener salli yate refus back donal...,0.405317,-0.9517,-172.81,0.758929
119057,pakistan air ambulance helicopter crash kills ...,1.0,266.0,10,10.000000,5,0,73,65,pakistan air ambul helicopt crash kill nine ht...,0.000000,-0.8126,46.44,1.000000
19524,add comment \nsome people fail to plan their ...,1.0,NaN,332,27.666667,22,3,1485,1446,add comment peopl fail plan purchas use firewo...,0.493783,-0.7893,-46.27,0.773050
2538,are you tired of supporting companies who are ...,1.0,NaN,391,27.928571,35,14,2121,2078,tire support compani care less futur nation ti...,0.460303,0.7845,-118.34,0.786096


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 124754 entries, 96862 to 123657
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   text                           124754 non-null  object 
 1   label                          124754 non-null  float64
 2   id                             7503 non-null    float64
 3   word_count                     124754 non-null  int32  
 4   avg_sentence_length            124751 non-null  float64
 5   text_punctuation_frequency     124754 non-null  int64  
 6   text_capitalization_frequency  124754 non-null  int32  
 7   text_before_character_removal  124754 non-null  int64  
 8   text_after_character_removal   124754 non-null  int64  
 9   text_processed                 124754 non-null  object 
 10  subjectivity_text              124754 non-null  float64
 11  text_sentiment                 124754 non-null  float64
 12  text_reading_ease              

In [32]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14583 entries, 0 to 14683
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   text                           14583 non-null  object 
 1   label                          14583 non-null  int64  
 2   word_count                     14583 non-null  int32  
 3   avg_sentence_length            14583 non-null  float64
 4   text_punctuation_frequency     14583 non-null  int64  
 5   text_capitalization_frequency  14583 non-null  int32  
 6   text_before_character_removal  14583 non-null  int64  
 7   text_after_character_removal   14583 non-null  int64  
 8   text_processed                 14583 non-null  object 
 9   subjectivity_text              14583 non-null  float64
 10  text_sentiment                 14583 non-null  float64
 11  text_reading_ease              14583 non-null  float64
 12  text_lexical_diversity         14583 non-null  floa

In [33]:
%%time
df_backup=df
df_test_backup=df_test
df_backup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 124754 entries, 96862 to 123657
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   text                           124754 non-null  object 
 1   label                          124754 non-null  float64
 2   id                             7503 non-null    float64
 3   word_count                     124754 non-null  int32  
 4   avg_sentence_length            124751 non-null  float64
 5   text_punctuation_frequency     124754 non-null  int64  
 6   text_capitalization_frequency  124754 non-null  int32  
 7   text_before_character_removal  124754 non-null  int64  
 8   text_after_character_removal   124754 non-null  int64  
 9   text_processed                 124754 non-null  object 
 10  subjectivity_text              124754 non-null  float64
 11  text_sentiment                 124754 non-null  float64
 12  text_reading_ease              

In [34]:
df_test_backup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14583 entries, 0 to 14683
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   text                           14583 non-null  object 
 1   label                          14583 non-null  int64  
 2   word_count                     14583 non-null  int32  
 3   avg_sentence_length            14583 non-null  float64
 4   text_punctuation_frequency     14583 non-null  int64  
 5   text_capitalization_frequency  14583 non-null  int32  
 6   text_before_character_removal  14583 non-null  int64  
 7   text_after_character_removal   14583 non-null  int64  
 8   text_processed                 14583 non-null  object 
 9   subjectivity_text              14583 non-null  float64
 10  text_sentiment                 14583 non-null  float64
 11  text_reading_ease              14583 non-null  float64
 12  text_lexical_diversity         14583 non-null  floa

In [35]:
%%time
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer with optional preprocessing options
vectorizer = CountVectorizer(
    stop_words='english',  # Remove common stop words
    max_features=1000,     # Limit the number of features to reduce dimensionality
    binary=True             # Use binary encoding (1 if word is present, 0 otherwise)
)

# Vectorize the text
X = vectorizer.fit_transform(df['text_processed'])

# Create a DataFrame with the BoW representation
bag_of_words_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Reset the index of df
df.reset_index(drop=True, inplace=True)

# Add a prefix to the BoW columns
new_column_names = ["BOW_" + col for col in bag_of_words_df.columns]
bag_of_words_df.columns = new_column_names

# Save the BoW DataFrame to a Parquet file
bag_of_words_df.to_parquet("BOW.parquet")

# Concatenate df and bag_of_words_df
concatenated_df = pd.concat([df, bag_of_words_df], axis=1)


CPU times: total: 32.3 s
Wall time: 33.8 s


In [36]:
%%time
X_test = vectorizer.transform(df_test['text_processed'])

# Create a DataFrame with the BoW representation for the test data
bag_of_words_test_df = pd.DataFrame(X_test.toarray(), columns=vectorizer.get_feature_names_out())
df_test.reset_index(drop=True, inplace=True)
# Add a prefix to the BoW columns for the test data
new_column_names_test = ["BOW_" + col for col in bag_of_words_test_df.columns]
bag_of_words_test_df.columns = new_column_names_test

# Concatenate the test_df and bag_of_words_test_df
concatenated_test_df = pd.concat([df_test, bag_of_words_test_df], axis=1)



CPU times: total: 2.78 s
Wall time: 3.07 s


In [37]:
df_test

,text,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal,text_processed,subjectivity_text,text_sentiment,text_reading_ease,text_lexical_diversity
0,trump promise new deal black america trump pro...,0,170,170.0,0,0,1187,1187,trump promis new deal black america trump prom...,0.547890,0.8658,-92.62,0.752941
1,foundation tie bedevil hillary clinton preside...,1,1087,1087.0,0,0,7994,7994,foundat tie bedevil hillari clinton presidenti...,0.310024,0.9666,-1030.82,0.494475
2,number week long russia end oil dependence rbt...,0,55,55.0,0,0,352,352,number week long russia end oil depend rbth ye...,0.338095,0.0772,32.57,0.800000
3,codesod rule ten remy porter remy escape enter...,0,124,124.0,0,0,1087,1087,codesod rule ten remi porter remi escap enterp...,0.261865,0.4404,-88.23,0.701613
4,switch chip know anymore home scitech switch c...,0,309,309.0,0,0,2141,2141,switch chip know anymor home scitech switch ch...,0.319414,0.9217,-242.17,0.728155
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14578,idiocy new york time editorial board scorch go...,1,184,184.0,0,0,1391,1391,idioci new york time editori board scorch gop ...,0.340720,-0.9349,-132.21,0.760870
14579,poll clinton lead trump slip florida shoot pol...,1,235,235.0,0,0,1721,1721,poll clinton lead trump slip florida shoot pol...,0.423810,-0.9908,-175.52,0.629787
14580,world war conspiracy episode november people c...,0,33,33.0,0,0,227,227,world war conspiraci episod novemb peopl crazi...,0.467708,-0.8316,37.98,0.969697
14581,parent american woman hold isi say notify deat...,1,395,395.0,0,0,2688,2688,parent american woman hold isi say notifi deat...,0.406771,0.9723,-328.44,0.571066


In [38]:
concatenated_test_df

,text,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal,text_processed,subjectivity_text,...,BOW_worri,BOW_worth,BOW_write,BOW_wrong,BOW_wrote,BOW_ye,BOW_year,BOW_yearold,BOW_york,BOW_young
0,trump promise new deal black america trump pro...,0,170,170.0,0,0,1187,1187,trump promis new deal black america trump prom...,0.547890,...,0,0,0,0,0,0,1,0,0,0
1,foundation tie bedevil hillary clinton preside...,1,1087,1087.0,0,0,7994,7994,foundat tie bedevil hillari clinton presidenti...,0.310024,...,1,0,1,0,0,0,1,0,1,0
2,number week long russia end oil dependence rbt...,0,55,55.0,0,0,352,352,number week long russia end oil depend rbth ye...,0.338095,...,0,0,0,0,0,0,1,0,0,0
3,codesod rule ten remy porter remy escape enter...,0,124,124.0,0,0,1087,1087,codesod rule ten remi porter remi escap enterp...,0.261865,...,0,0,1,0,0,0,0,0,0,0
4,switch chip know anymore home scitech switch c...,0,309,309.0,0,0,2141,2141,switch chip know anymor home scitech switch ch...,0.319414,...,0,0,1,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14578,idiocy new york time editorial board scorch go...,1,184,184.0,0,0,1391,1391,idioci new york time editori board scorch gop ...,0.340720,...,0,0,1,0,0,0,0,0,1,0
14579,poll clinton lead trump slip florida shoot pol...,1,235,235.0,0,0,1721,1721,poll clinton lead trump slip florida shoot pol...,0.423810,...,0,0,0,0,0,0,1,0,0,0
14580,world war conspiracy episode november people c...,0,33,33.0,0,0,227,227,world war conspiraci episod novemb peopl crazi...,0.467708,...,0,0,0,0,0,0,0,0,0,0
14581,parent american woman hold isi say notify deat...,1,395,395.0,0,0,2688,2688,parent american woman hold isi say notifi deat...,0.406771,...,0,0,1,0,0,0,1,1,0,1


In [39]:
#Best of 5 text
bag_of_words_df.shape
bag_of_words_df.head()
#compund words

,BOW_abil,BOW_abl,BOW_absolut,BOW_abus,BOW_accept,BOW_access,BOW_accord,BOW_account,BOW_accus,BOW_achiev,...,BOW_worri,BOW_worth,BOW_write,BOW_wrong,BOW_wrote,BOW_ye,BOW_year,BOW_yearold,BOW_york,BOW_young
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124754 entries, 0 to 124753
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   text                           124754 non-null  object 
 1   label                          124754 non-null  float64
 2   id                             7503 non-null    float64
 3   word_count                     124754 non-null  int32  
 4   avg_sentence_length            124751 non-null  float64
 5   text_punctuation_frequency     124754 non-null  int64  
 6   text_capitalization_frequency  124754 non-null  int32  
 7   text_before_character_removal  124754 non-null  int64  
 8   text_after_character_removal   124754 non-null  int64  
 9   text_processed                 124754 non-null  object 
 10  subjectivity_text              124754 non-null  float64
 11  text_sentiment                 124754 non-null  float64
 12  text_reading_ease             

In [42]:
%%time

df.to_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\ForModeling.parquet")

concatenated_df.to_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\ForModeling_BagofWords.parquet")

df.iloc[:, :-1000]

df.to_csv(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Visualisation\Visualize.csv")



CPU times: total: 16.9 s
Wall time: 18.8 s


In [43]:
df_test.to_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\ForModeling_test.parquet")

concatenated_test_df.to_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\ForModeling_BagofWords_test.parquet")